In [4]:
## A guy asked me to help him with recognition of car numbers using computer vision



In [ ]:
##His version

import cv2
import numpy as np

# Захват видеопотока с камеры (или из файла)
cap = cv2.VideoCapture(0)  # 0 для использования встроенной камеры, либо путь к видеофайлу

while True:
    # Захват кадра из видеопотока
    ret, frame = cap.read()

    # Предобработка кадра, если необходимо
    # ...

    # Применение обработки изображения для распознавания номера автомобиля
    # ...
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 3), np.uint8)
    morphology = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    edged = cv2.Canny(binary, 50, 150)
    gray1 = cv2.GaussianBlur(gray, (5, 5), 0)
    
# Применение методов обнаружения краев (например, Canny)
    edged = cv2.Canny(gray1, 50, 150)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    possible_plates = []


    for contour in contours:
        # Оцениваем размер и форму контура
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if (((3 < aspect_ratio < 7) and ((w > 200)))):  # Подбираем соотношение сторон для номерного знака
            possible_plates.append(contour)

    # Рисуем найденные контуры (потенциальные номерные знаки) на исходном изображении
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.drawContours(frame, possible_plates, -1, (0, 255, 0), 3)
    cv2.imshow(' Ie',possible_plates)

    # Отображение обработанного кадра
    cv2.imshow('Preprocessed Image', morphology)
    cv2.imshow(' Image-gray', gray)
    cv2.imshow(' Image',frame)



    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

In [ ]:
##Mine final cut

conda install -c menpo opencv
import cv2
import numpy as np
import mysql.connector
import serial

# Подключение к базе данных
db = mysql.connector.connect(
    host="localhost",
    user="username",
    password="password",
    database="database_name"
)

# Инициализация подключения к ардуино
arduino = serial.Serial('COM3', 9600)  # Подставьте соответствующий порт и скорость

# Захват видеопотока с камеры (или из файла)
cap = cv2.VideoCapture(0)  # 0 для использования встроенной камеры, либо путь к видеофайлу

while True:
    # Захват кадра из видеопотока
    ret, frame = cap.read()

    # Предобработка кадра, если необходимо
    # ...

    # Применение обработки изображения для распознавания номера автомобиля
    # ...
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 3), np.uint8)
    morphology = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    edged = cv2.Canny(binary, 50, 150)
    gray1 = cv2.GaussianBlur(gray, (5, 5), 0)

    # Применение методов обнаружения краев (например, Canny)
    edged = cv2.Canny(gray1, 50, 150)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    possible_plates = []

    for contour in contours:
        # Оцениваем размер и форму контура
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 3 < aspect_ratio < 7 and w > 200:  # Подбираем соотношение сторон для номерного знака
            possible_plates.append(contour)

    # Рисуем найденные контуры (потенциальные номерные знаки) на исходном изображении
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.drawContours(frame, possible_plates, -1, (0, 255, 0), 3)
    cv2.imshow('Plate Detection', frame)

    # Получение номеров соответствующих контуров
    found_plates = []
    for i, plate in enumerate(possible_plates):
        x, y, w, h = cv2.boundingRect(plate)
        plate_img = frame[y:y + h, x:x + w]
        # Обработка изображения номера (например, распознавание символов)
        # ...
        # Получение строки с номером
        plate_number = "AB123CD"  # Замените на код распознавания номеров

        # Поиск номера в базе данных
        cursor = db.cursor()
        cursor.execute("SELECT * FROM license_plates WHERE plate_number = %s", (plate_number,))
        result = cursor.fetchone()
        if result:
            found_plates.append(plate_number)

    # Отправка сигнала на ардуино для запуска сервопривода
    if found_plates:
        arduino.write(b'1')  # Отправляем сигнал на ардуино

    # Отображение обработанного кадра
    cv2.imshow('Preprocessed Image', morphology)
    cv2.imshow(' Edged-Display', edged)

    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
db.close()